## Golang

In [24]:
import "net/http"

// https://the-odds-api.com/
var API_KEY = "GET_YOUR_OWN"

func getSportsResponse() (*http.Response, error){
    var res *http.Response
    var err error

    client := &http.Client{}

    req, err := http.NewRequest("GET", "https://api.the-odds-api.com/v3/sports", nil)
    if err != nil {
        return res, err
    }

    q := req.URL.Query()
    q.Add("api_key", API_KEY)
    req.URL.RawQuery = q.Encode()
    
    return client.Do(req)
}


In [25]:
res, err := getSportsResponse()

In [26]:
res

&{200 OK 200 HTTP/2.0 2 0 map[X-Requests-Used:[3] X-Amzn-Trace-Id:[Root=1-5edc7bab-595211b1b163ce3b5573478d;Sampled=0] X-Cache:[Miss from cloudfront] Via:[1.1 0376479bf82ad53b72305db1fb2a28c8.cloudfront.net (CloudFront)] X-Amzn-Requestid:[88183598-5ca0-4b97-a2ae-9fa8fac098b0] X-Requests-Remaining:[497] X-Amzn-Remapped-Content-Length:[3800] Date:[Sun, 07 Jun 2020 05:31:24 GMT] Access-Control-Allow-Origin:[*] X-Amz-Apigw-Id:[NvhC4FA-ywMF-Aw=] Access-Control-Expose-Headers:[X-Requests-Remaining, X-Requests-Used] X-Amz-Cf-Pop:[SYD4-C1] Content-Type:[application/json] X-Amz-Cf-Id:[TOSsqiU3Ttrd9cyDOdiu5EUv1blT_yY-BxH17v7rYw0LqxkfJlXu0A==]] 0xc00027df20 -1 [] false true map[] 0xc0000b8c00 0xc0000da6e0}

In [27]:
import "io/ioutil"

data, _ := ioutil.ReadAll(res.Body)

In [30]:
string(data)

{"success":true,"data":[{"key":"americanfootball_ncaaf","active":true,"group":"American Football","details":"US College Football","title":"NCAAF","has_outrights":false},{"key":"americanfootball_nfl","active":true,"group":"American Football","details":"US Football","title":"NFL","has_outrights":false},{"key":"aussierules_afl","active":true,"group":"Aussie Rules","details":"Aussie Football","title":"AFL","has_outrights":false},{"key":"cricket_test_match","active":true,"group":"Cricket","details":"International Test Matches","title":"Test Matches","has_outrights":false},{"key":"esports_lol","active":true,"group":"Esports","details":"LCS, LEC, LCK, LPL","title":"League of Legends","has_outrights":false},{"key":"mma_mixed_martial_arts","active":true,"group":"Mixed Martial Arts","details":"Mixed Martial Arts","title":"MMA","has_outrights":false},{"key":"rugbyleague_nrl","active":true,"group":"Rugby League","details":"Aussie Rugby League","title":"NRL","has_outrights":false},{"key":"soccer_de

In [31]:
// https://mholt.github.io/json-to-go/
type Sports struct {
    Success bool `json:"success"`
    Data    []struct {
        Key          string `json:"key"`
        Active       bool   `json:"active"`
        Group        string `json:"group"`
        Details      string `json:"details"`
        Title        string `json:"title"`
        HasOutrights bool   `json:"has_outrights"`
    } `json:"data"`
}

In [33]:
import "encoding/json"

sports := Sports{}
err := json.Unmarshal(data, &sports)
err

In [34]:
sports

{true [{americanfootball_ncaaf true American Football US College Football NCAAF false} {americanfootball_nfl true American Football US Football NFL false} {aussierules_afl true Aussie Rules Aussie Football AFL false} {cricket_test_match true Cricket International Test Matches Test Matches false} {esports_lol true Esports LCS, LEC, LCK, LPL League of Legends false} {mma_mixed_martial_arts true Mixed Martial Arts Mixed Martial Arts MMA false} {rugbyleague_nrl true Rugby League Aussie Rugby League NRL false} {soccer_denmark_superliga true Soccer - Europe Danish Soccer 🇩🇰 Denmark Superliga false} {soccer_epl true Soccer - UK English Premier League 🇬🇧 EPL false} {soccer_fa_cup true Soccer - UK English FA Cup 🇬🇧 FA Cup false} {soccer_germany_bundesliga true Soccer - Europe German Soccer 🇩🇪 Bundesliga - Germany false} {soccer_germany_bundesliga2 true Soccer - Europe German Soccer 🇩🇪 Bundesliga 2 - Germany false} {soccer_italy_serie_a true Soccer - Europe Italian Soccer 🇮🇹 Serie A - Italy fals

In [35]:
sports.Success

true

In [36]:
sports.Data[0]

{americanfootball_ncaaf true American Football US College Football NCAAF false}

In [37]:
sports.Data[0].Key

americanfootball_ncaaf

In [39]:
// putting this together for processing the sports response
import (
    "encoding/json"
    "io/ioutil"
)

type Sports struct {
    Success bool `json:"success"`
    Data    []struct {
        Key          string `json:"key"`
        Active       bool   `json:"active"`
        Group        string `json:"group"`
        Details      string `json:"details"`
        Title        string `json:"title"`
        HasOutrights bool   `json:"has_outrights"`
    } `json:"data"`
}

func getSports() (Sports, error){
    sports := Sports{}
    
    res, err := getSportsResponse()
    if err != nil {
        return sports, err
    }

    data, err := ioutil.ReadAll(res.Body)
    if err != nil {
        return sports, err
    }

    err := json.Unmarshal(data, &sports)
    return sports, err
}

In [40]:
sports, err := getSports()

In [42]:
sports.Success

true

Use this sports struct to extract keys...

In [66]:
var keys []string

for _, sport := range sports.Data {
    keys = append(keys, sport.Key)
}

keys

[americanfootball_ncaaf americanfootball_nfl aussierules_afl cricket_test_match esports_lol mma_mixed_martial_arts rugbyleague_nrl soccer_denmark_superliga soccer_epl soccer_fa_cup soccer_germany_bundesliga soccer_germany_bundesliga2 soccer_italy_serie_a soccer_korea_kleague1 soccer_norway_eliteserien soccer_portugal_primeira_liga soccer_russia_premier_league soccer_spain_la_liga soccer_spain_segunda_division soccer_sweden_allsvenskan soccer_sweden_superettan soccer_switzerland_superleague soccer_turkey_super_league]

Now get the odds for a particular sport.

In [73]:
func getSportOddsResponse(sportKey string) (*http.Response, error){
    var res *http.Response
    var err error

    client := &http.Client{}

    req, err := http.NewRequest("GET", "https://api.the-odds-api.com/v3/odds", nil)
    if err != nil {
        return res, err
    }

    q := req.URL.Query()
    q.Add("api_key", API_KEY)
    q.Add("sport", sportKey)
    q.Add("region", "uk")  // uk | us | eu | au
    q.Add("mkt", "h2h")    // h2h | spreads | totals
    req.URL.RawQuery = q.Encode()
    
    return client.Do(req)
}


In [74]:
res, err := getSportOddsResponse("soccer_spain_segunda_division")

In [ ]:
data, _ := ioutil.ReadAll(res.Body)
string(data)

In [100]:
type AutoGenerated struct {
    Success bool `json:"success"`
    Data    []struct {
        SportKey     string   `json:"sport_key"`
        SportNice    string   `json:"sport_nice"`
        Teams        []string `json:"teams"`
        CommenceTime int      `json:"commence_time"`
        HomeTeam     string   `json:"home_team"`
        Sites        []struct {
            SiteKey    string `json:"site_key"`
            SiteNice   string `json:"site_nice"`
            LastUpdate int    `json:"last_update"`
            Odds       struct {
                H2H []float64 `json:"h2h"`
            } `json:"odds,omitempty"`
            OddsLay struct {
                H2H    []float64 `json:"h2h"`
                H2HLay []float64 `json:"h2h_lay"`
            } `json:"odds,omitempty"`
        } `json:"sites"`
        SitesCount int `json:"sites_count"`
    } `json:"data"`
}


In [98]:
func getSportOdds(sportKey string) (SportOdds, error){
    sportOdds := SportOdds{}

    res, err := getSportOddsResponse(sportKey)
    if err != nil {
        return sportOdds, err
    }

    data, err := ioutil.ReadAll(res.Body)
    if err != nil {
        return sportOdds, err
    }

    err := json.Unmarshal(data, &sportOdds)
    return sportOdds, err
}

In [101]:
sportOdds, err := getSportOdds("soccer_spain_segunda_division")

In [102]:
sportOdds.Success

true

In [109]:
first := sportOdds.Data[0]
first

{soccer_spain_segunda_division La Liga 2 - Spain [Málaga SD Huesca] 1591983000 Málaga [{onexbet 1xBet 1591509773 {[2.62 2.9 3.12]}} {paddypower Paddy Power 1591509937 {[2.4 2.8 3]}} {williamhill William Hill 1591509443 {[2.55 2.88 3]}} {sport888 888sport 1591509930 {[2.48 2.8 3]}} {unibet Unibet 1591509925 {[2.48 2.8 3]}} {betvictor Bet Victor 1591509963 {[2.6 3 3]}} {marathonbet Marathon Bet 1591509893 {[2.6 2.91 3.12]}} {skybet Sky Bet 1591509783 {[2.45 2.7 2.9]}} {betfair Betfair 1591509893 {[1.01 2.72 2.86]}} {matchbook Matchbook 1591509909 {[2.15 2.4 2.5]}}] 10}

In [110]:
first.SportKey

soccer_spain_segunda_division

In [111]:
first.Sites

[{onexbet 1xBet 1591509773 {[2.62 2.9 3.12]}} {paddypower Paddy Power 1591509937 {[2.4 2.8 3]}} {williamhill William Hill 1591509443 {[2.55 2.88 3]}} {sport888 888sport 1591509930 {[2.48 2.8 3]}} {unibet Unibet 1591509925 {[2.48 2.8 3]}} {betvictor Bet Victor 1591509963 {[2.6 3 3]}} {marathonbet Marathon Bet 1591509893 {[2.6 2.91 3.12]}} {skybet Sky Bet 1591509783 {[2.45 2.7 2.9]}} {betfair Betfair 1591509893 {[1.01 2.72 2.86]}} {matchbook Matchbook 1591509909 {[2.15 2.4 2.5]}}]

In [112]:
first.Sites[0]

{onexbet 1xBet 1591509773 {[2.62 2.9 3.12]}}

In [113]:
first.Sites[0].Odds

{[2.62 2.9 3.12]}

***